# Metin Sınıflandırma Görevi

Bu modülde, **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** veri setine dayalı basit bir metin sınıflandırma göreviyle başlayacağız: haber başlıklarını Dünya, Spor, İş ve Bilim/Teknoloji olmak üzere 4 kategoriden birine sınıflandıracağız.

## Veri Seti

Veri setini yüklemek için **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** API'sini kullanacağız.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Artık veri kümesinin eğitim ve test bölümlerine sırasıyla `dataset['train']` ve `dataset['test']` kullanarak erişebiliriz:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Haydi veri setimizden ilk 10 yeni başlığı yazdıralım:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Metin Vektörleştirme

Şimdi metni tensörler olarak temsil edilebilecek **sayılar**a dönüştürmemiz gerekiyor. Eğer kelime düzeyinde bir temsil istiyorsak, iki şey yapmamız gerekiyor:

* Metni **token**lara ayırmak için bir **tokenizer** kullanmak.
* Bu tokenların bir **vocab**ını oluşturmak.

### Kelime dağarcığı boyutunu sınırlama

AG News veri seti örneğinde, kelime dağarcığı boyutu oldukça büyük, 100 binden fazla kelime içeriyor. Genel olarak konuşursak, metinde nadiren bulunan kelimelere ihtiyacımız yok — sadece birkaç cümlede yer alacaklar ve model bu kelimelerden öğrenemeyecek. Bu nedenle, vektörleştirici yapıcısına bir argüman geçirerek kelime dağarcığı boyutunu daha küçük bir sayıya sınırlamak mantıklı olacaktır:

Bu adımların her ikisi de **TextVectorization** katmanı kullanılarak gerçekleştirilebilir. Şimdi vektörleştirici nesnesini oluşturalım ve ardından `adapt` metodunu çağırarak tüm metni gözden geçirip bir kelime dağarcığı oluşturalım:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Not** yalnızca tüm veri setinin bir alt kümesini kullanarak bir kelime dağarcığı oluşturduğumuzu unutmayın. Bunu, işlem süresini hızlandırmak ve sizi bekletmemek için yapıyoruz. Ancak, tüm veri setindeki bazı kelimelerin kelime dağarcığına dahil edilmeme ve eğitim sırasında göz ardı edilme riskini alıyoruz. Bu nedenle, `adapt` sırasında tüm kelime dağarcığı boyutunu kullanmak ve tüm veri setinden geçmek nihai doğruluğu artırabilir, ancak bu artış önemli olmayacaktır.

Şimdi gerçek kelime dağarcığına erişebiliriz:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Vektörleştiriciyi kullanarak, herhangi bir metni kolayca bir sayı kümesine kodlayabiliriz:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Bag-of-words metin temsili

Kelimeler anlamı temsil ettiği için, bazen bir metnin anlamını cümledeki sıralarına bakmaksızın sadece bireysel kelimelere bakarak anlayabiliriz. Örneğin, haberleri sınıflandırırken, *hava durumu* ve *kar* gibi kelimeler muhtemelen *hava durumu tahmini*ni işaret ederken, *hisse senetleri* ve *dolar* gibi kelimeler *finans haberleri*ne işaret edecektir.

**Bag-of-words** (BoW) vektör temsili, anlaması en basit geleneksel vektör temsilidir. Her kelime bir vektör indeksine bağlanır ve bir vektör elemanı, belirli bir belgede her kelimenin kaç kez geçtiğini içerir.

![Bag-of-words vektör temsilinin bellekte nasıl temsil edildiğini gösteren bir görsel.](../../../../../translated_images/tr/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Not**: BoW'yu, metindeki bireysel kelimeler için tekil olarak birleştirilmiş tüm one-hot-encoded vektörlerin toplamı olarak da düşünebilirsiniz.

Aşağıda, Scikit Learn Python kütüphanesini kullanarak bir bag-of-words temsili oluşturmanın bir örneği verilmiştir:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Yukarıda tanımladığımız Keras vektörleştiricisini de kullanabiliriz, her kelime numarasını bir one-hot kodlamasına dönüştürerek ve tüm bu vektörleri toplayarak:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Not**: Sonuçların önceki örnekten farklı olmasına şaşırabilirsiniz. Bunun nedeni, Keras örneğinde vektörün uzunluğunun, tüm AG News veri setinden oluşturulan kelime dağarcığı boyutuna karşılık gelmesidir. Ancak Scikit Learn örneğinde kelime dağarcığını örnek metinden anlık olarak oluşturduk.


## BoW sınıflandırıcısını eğitmek

Artık metnimizin kelime torbası (bag-of-words) temsilini nasıl oluşturacağımızı öğrendiğimize göre, bunu kullanan bir sınıflandırıcı eğitelim. Öncelikle, veri setimizi kelime torbası temsiline dönüştürmemiz gerekiyor. Bunu aşağıdaki şekilde `map` fonksiyonunu kullanarak gerçekleştirebiliriz:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Şimdi bir doğrusal katman içeren basit bir sınıflandırıcı sinir ağı tanımlayalım. Girdi boyutu `vocab_size` ve çıktı boyutu sınıf sayısına (4) karşılık gelir. Bir sınıflandırma görevi çözdüğümüz için, son aktivasyon fonksiyonu **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Dört sınıfımız olduğu için, %80'in üzerinde bir doğruluk iyi bir sonuçtur.

## Bir sınıfı tek bir ağ olarak eğitmek

Vektörleştirici aynı zamanda bir Keras katmanı olduğu için, onu içeren bir ağ tanımlayabilir ve uçtan uca eğitebiliriz. Bu şekilde, veri kümesini `map` kullanarak vektörleştirmemize gerek kalmaz, sadece orijinal veri kümesini ağın girişine iletebiliriz.

> **Not**: Yine de veri kümesindeki alanları (örneğin `title`, `description` ve `label`) sözlüklerden tuple'lara dönüştürmek için `map` uygulamamız gerekecek. Ancak, verileri diskten yüklerken, ilk etapta gerekli yapıya sahip bir veri kümesi oluşturabiliriz.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## İkili, üçlü ve n-gramlar

Bag-of-words yaklaşımının bir sınırlaması, bazı kelimelerin çok kelimeli ifadelerin bir parçası olmasıdır. Örneğin, 'hot dog' kelimesi, diğer bağlamlarda 'hot' ve 'dog' kelimelerinden tamamen farklı bir anlama sahiptir. Eğer 'hot' ve 'dog' kelimelerini her zaman aynı vektörlerle temsil edersek, bu modelimizi yanıltabilir.

Bu durumu ele almak için, **n-gram temsilleri** genellikle belge sınıflandırma yöntemlerinde kullanılır. Burada, her bir kelimenin, iki kelimelik veya üç kelimelik ifadelerin sıklığı, sınıflandırıcıları eğitmek için faydalı bir özellik olarak kullanılır. Örneğin, ikili (bigram) temsillerde, orijinal kelimelere ek olarak tüm kelime çiftlerini de kelime dağarcığına ekleriz.

Aşağıda, Scikit Learn kullanarak bir ikili bag-of-words temsili oluşturmanın bir örneği verilmiştir:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

n-gram yaklaşımının en büyük dezavantajı, kelime dağarcığının boyutunun çok hızlı bir şekilde büyümeye başlamasıdır. Pratikte, n-gram temsilini bir boyut azaltma tekniğiyle, örneğin *gömüler* ile birleştirmemiz gerekir. Bu konuyu bir sonraki birimde ele alacağız.

**AG News** veri setimizde bir n-gram temsili kullanmak için, `ngrams` parametresini `TextVectorization` yapıcımıza geçirmemiz gerekiyor. Bir bigram kelime dağarcığının uzunluğu **önemli ölçüde daha büyüktür**, bizim durumumuzda bu sayı 1,3 milyondan fazla token! Bu nedenle, bigram tokenlerini de makul bir sayıyla sınırlamak mantıklıdır.

Sınıflandırıcıyı eğitmek için yukarıdaki kodun aynısını kullanabiliriz, ancak bu çok bellek verimsiz olur. Bir sonraki birimde, gömüler kullanarak bigram sınıflandırıcıyı eğiteceğiz. Bu arada, bu not defterinde bigram sınıflandırıcı eğitimiyle deney yapabilir ve daha yüksek bir doğruluk elde edip edemeyeceğinizi görebilirsiniz.


## BoW Vektörlerini Otomatik Hesaplama

Yukarıdaki örnekte, BoW vektörlerini bireysel kelimelerin tekil kodlamalarını toplayarak elle hesapladık. Ancak, TensorFlow'un en son sürümü, vektörleştirici yapıcısına `output_mode='count` parametresini geçirerek BoW vektörlerini otomatik olarak hesaplamamıza olanak tanıyor. Bu, modelimizi tanımlamayı ve eğitmeyi önemli ölçüde kolaylaştırıyor:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Terim Frekansı - Ters Doküman Frekansı (TF-IDF)

BoW (Bag-of-Words) temsilinde, kelime geçişleri, kelimenin kendisinden bağımsız olarak aynı teknikle ağırlıklandırılır. Ancak, *bir* ve *içinde* gibi sık kullanılan kelimelerin sınıflandırma için özel terimlere kıyasla çok daha az önemli olduğu açıktır. Çoğu NLP görevinde bazı kelimeler diğerlerinden daha önemlidir.

**TF-IDF**, **terim frekansı - ters doküman frekansı** anlamına gelir. Bu, bag-of-words'un bir varyasyonudur; burada bir kelimenin bir dokümanda görünüp görünmediğini belirten ikili 0/1 değeri yerine, kelimenin korpus içindeki geçiş sıklığıyla ilişkili bir kayan nokta değeri kullanılır.

Daha resmi olarak, bir kelimenin $i$ doküman $j$ içindeki ağırlığı $w_{ij}$ şu şekilde tanımlanır:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
burada
* $tf_{ij}$, $i$ kelimesinin $j$ dokümanındaki geçiş sayısıdır, yani daha önce gördüğümüz BoW değeri
* $N$, koleksiyondaki doküman sayısıdır
* $df_i$, $i$ kelimesini içeren dokümanların tüm koleksiyondaki sayısıdır

TF-IDF değeri $w_{ij}$, bir kelimenin bir dokümanda kaç kez geçtiğiyle orantılı olarak artar ve kelimenin korpus içindeki dokümanlarda geçiş sayısıyla dengelenir. Bu, bazı kelimelerin diğerlerinden daha sık geçtiği gerçeğini ayarlamaya yardımcı olur. Örneğin, eğer bir kelime koleksiyondaki *her* dokümanda geçiyorsa, $df_i=N$ olur ve $w_{ij}=0$ olur; bu terimler tamamen göz ardı edilir.

Scikit Learn kullanarak metnin TF-IDF vektörleştirmesini kolayca oluşturabilirsiniz:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

Keras'ta, `TextVectorization` katmanı, `output_mode='tf-idf'` parametresini geçirerek otomatik olarak TF-IDF frekanslarını hesaplayabilir. TF-IDF kullanmanın doğruluğu artırıp artırmadığını görmek için yukarıda kullandığımız kodu tekrar edelim:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Sonuç

TF-IDF temsilleri farklı kelimelere sıklık ağırlıkları verse de, anlamı veya sıralamayı temsil edemezler. Ünlü dilbilimci J. R. Firth'in 1935'te söylediği gibi, "Bir kelimenin tam anlamı her zaman bağlamsaldır ve bağlamdan bağımsız bir anlam çalışması ciddiye alınamaz." Kursun ilerleyen bölümlerinde dil modelleme kullanarak metinden bağlamsal bilgiyi nasıl yakalayacağımızı öğreneceğiz.



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul etmiyoruz.
